<a href="https://colab.research.google.com/github/kotharisanjana/CMPE297_SpecialTopics_Fall2023/blob/main/Assignment_4/Function_Calling_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [5]:
import openai
import requests

In [3]:
openai.api_key = ""

In [4]:
model = "gpt-3.5-turbo-0613"

In [6]:
def chat_completion_request(messages, functions, model=model):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }

    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})

    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
# functions = [
#     {
#       "name": "book_travel",
#       "description": "Book travel",
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "destination": {
#             "type": "string",
#             "description": "Your travel destination."
#           },
#           "departute": {
#             "type": "string",
#             "description": "From where are you traveling"
#           },
#           "number_people": {
#             "type": "string",
#             "description": "How many people are traveling"
#             },
#           "travel_mode": {
#             "type": "string",
#             "description": "What mode of travel will it be."
#           }
#         }
#       }
#     }
#   ]

In [7]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
    {
        "name": "get_n_day_weather_forecast",
        "description": "Get an N-day weather forecast",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
                "num_days": {
                    "type": "integer",
                    "description": "The number of days to forecast",
                }
            },
            "required": ["location", "format", "num_days"]
        },
    },
]

In [8]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': 'Sure, I can help you with that. Can you please provide me with the name of the city you would like to know the weather for?'}

In [9]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}'}}

In [10]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': 'Sure, for how many days would you like to get the weather forecast?'}

In [11]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, functions=functions
)
chat_response.json()["choices"][0]

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_n_day_weather_forecast',
   'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}'}},
 'finish_reason': 'function_call'}

In [14]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': "I'm sorry, could you please tell me the location for which you would like to know the weather?"}